# Connecting pyspark with csv

In [2]:
import pyspark

In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

In [4]:
spark=SparkSession.builder.appName('medicalsurvey').getOrCreate()
spark

KeyboardInterrupt: 

In [7]:
df_pyspark = spark.read.option('header','true').csv(r'data.csv',inferSchema=True)
data = pd.read_csv(r'data.csv')
df_pyspark

NameError: name 'spark' is not defined

In [2]:
df_pyspark.describe().show()

NameError: name 'df_pyspark' is not defined

In [3]:
df_pyspark.describe().show()

NameError: name 'df_pyspark' is not defined

In [4]:
df_pyspark.printSchema()

NameError: name 'df_pyspark' is not defined

In [5]:
z=df_pyspark.withColumn("anamoly",df_pyspark["heartbeat"]>90)
z.printSchema()

NameError: name 'df_pyspark' is not defined

# Streaming and Ploting data in real time

In [ ]:
dataschema=z.schema
dataschema

In [ ]:
streaming=(
    spark.readStream.schema(dataschema).option("maxFilesPerTrigger",1).csv(r'data.csv')
)
streaming.writeStream

In [ ]:
querry=(streaming.writeStream.queryName("table")#streaming
         .format("memory")#parquet
         #.outputMode("complete")
        .start()
        )

In [ ]:
import time

In [ ]:
df_pyspark.createOrReplaceTempView("table1")
#df_pyspark.filter("heartbeat>90")
df2 = spark.sql("SELECT time_span , heartbeat  from table1")
mong = df2.toPandas()
for row in df2.collect():
    print(row)
    #time.sleep(0.01)

In [ ]:
startdate=data.time_span[:1]
enddate=data.time_span[-1:]

In [ ]:
for i in range(50):
    df_pyspark.createOrReplaceTempView("table1")
    #df_pyspark.filter("heartbeat>90")
    df2 = spark.sql("SELECT time_span , heartbeat  from table1")
    df2.show(i)
import dash
import time
import dash_html_components as html
import dash_core_components as dcc
import numpy as np
from dash.dependencies import Input, Output
resolution =100000
x=data["time_span"]
y=data["heartbeat"]
startdate=data.time_span[:1]
figure= dict(data=[{'x': [], 'y': []}],layout=dict(xaxis=dict(range=[startdate, enddate]),yaxis=dict(range=[80, 100])))
app = dash.Dash(__name__, update_title="Heartbeat")
app.layout = html.Div([dcc.Graph(id='graph', figure=figure), dcc.Interval(id="interval")])
@app.callback(Output('graph', 'extendData'), [Input('interval', 'n_intervals')])
def update_data(n_intervals):
    index = n_intervals % resolution

    # tuple is (dict of new data, target trace index, number of points to keep)
    return dict(x=[[x[index]]], y=[[y[index]]])
if __name__ == '__main__':
    app.run_server(proxy=False,debug=False)
    

# Storing data into database

In [ ]:
import pandas as pd
import pymongo
import json

client = pymongo.MongoClient("mongodb://localhost:27017")
data = mong.to_dict(orient = "records")
db = client["MedicalAnamoly"]
db.HeartBeat1data.insert_many(data)